## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
import os
os.chdir("D:\CAIR, DRDO Internship 2021\Andrew Ng Machine Learning\machine-learning-ex1\ex1")

In [3]:
#Load data 

data = open("ex1data1.txt","r")
d = [n.split(",") for n in data]

data_df = pd.DataFrame(d)
df = data_df.replace("\n","",regex = True)
df.head()

,0,1
0,6.1101,17.592
1,5.5277,9.1302
2,8.5186,13.662
3,7.0032,11.854
4,5.8598,6.8233


In [4]:
#Input Data

inputs = df.loc[:,0]
inputs.head()

0    6.1101
1    5.5277
2    8.5186
3    7.0032
4    5.8598
Name: 0, dtype: object

In [5]:
#Converting dataframe to numpy array

input_arr1 = inputs.to_numpy()
input_arr = input_arr1.astype(np.float)

In [6]:
#Targets

targets = df.loc[:,1]
targets.head()

0    17.592
1    9.1302
2    13.662
3    11.854
4    6.8233
Name: 1, dtype: object

In [7]:
#Converting dataframe to numpy array

target_arr1 = targets.to_numpy()
target_arr = target_arr1.astype(np.float)

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [8]:
# Convert inputs and targets to tensors

inputs_tensor = torch.from_numpy(input_arr)
target_tensor = torch.from_numpy(target_arr)
inputs_tensor.shape
#print(inputs_tensor)
#print(target_tensor)

torch.Size([97])

In [9]:
import matplotlib.pyplot as plt

plt.plot(inputs_tensor, target_tensor,".")
plt.xlabel("Population of City in 10,000s")
plt.ylabel("Profit in $10,000s")

Text(0, 0.5, 'Profit in $10,000s')

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [10]:
# Weights and biases

w = torch.randn(1,inputs_tensor.shape[0], requires_grad=True)
b = torch.randn(1,inputs_tensor.shape[0] , requires_grad=True)
print(w)
print(b)

tensor([[-0.4477,  0.2369,  0.8880, -0.6696,  1.2064,  2.8565,  1.1719, -0.8014,
          1.1286, -0.0455, -0.0478, -2.0641,  0.5735,  0.2469,  0.3314, -1.4978,
          0.1663,  0.0702,  0.7890, -1.4708, -1.1601,  1.3935,  0.8092,  0.5814,
          0.8074,  0.5682,  0.0778,  0.0995, -0.2982,  1.1042, -0.5931,  0.4432,
         -1.1230,  0.2500,  0.5983,  0.4197, -1.3638, -0.0213,  0.3727, -1.1828,
          1.1756,  0.0996, -0.5443,  0.2518, -0.0046, -0.2861, -2.1497,  0.9983,
         -0.0464,  0.6934, -0.3807,  0.9900, -0.9996,  1.4707,  1.4489, -0.3441,
          0.3610,  0.5304,  0.9306,  2.6264, -3.3373, -0.4624, -0.8587, -0.6527,
         -0.7350,  0.4743, -1.3963,  0.4141, -0.2004,  0.2807,  0.5454, -0.4717,
         -0.1470, -3.1968, -1.0490, -0.8867, -1.8133,  0.9407, -0.5100, -1.2234,
         -1.9143, -0.3672, -1.1506,  0.3424, -2.7108,  0.7486, -0.3076, -0.2460,
         -1.4921, -0.2684, -1.3765,  0.7667, -0.3086, -0.3279,  2.0052,  0.9371,
          2.0364]], requires

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [11]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [12]:
# Generate predictions

preds = model(inputs_tensor.float())
print(preds)

tensor([[-23.4282, -23.2356, -21.7913, -24.2880, -21.9697, -23.7786, -24.0495,
         -25.4024, -25.5948, -25.2489, -23.3441, -24.3661, -22.5732, -25.6472,
         -23.4781, -24.4101, -23.6382, -23.0866, -24.0979, -24.1650, -25.1110,
         -25.2533, -23.4150, -25.0306, -22.4235, -22.6525, -24.0122, -23.9006,
         -25.8764, -23.6325, -24.5610, -25.3649, -24.5038, -24.0397, -24.6226,
         -21.6098, -26.4041, -25.3002, -24.2571, -24.5396, -22.7588, -24.3031,
         -25.0692, -23.7149, -24.3778, -22.7101, -23.2254, -22.9932, -22.1865,
         -24.0849, -24.3239, -23.7174, -24.7390, -22.6390, -23.6795, -24.1746,
         -24.2992, -24.0262, -23.4501, -23.6489, -26.2294, -25.4322, -24.5256,
         -24.2701, -22.3434, -22.9538, -24.2879, -24.3823, -24.7398, -24.6903,
         -25.8646, -22.5944, -24.8306, -23.1394, -25.6195, -24.0874, -23.9043,
         -25.6842, -23.8151, -23.4089, -24.9481, -23.0157, -22.3214, -22.4534,
         -24.3684, -23.7032, -23.8678, -25.4932, -24

Let's compare the predictions of our model with the actual targets.

In [13]:
# Compare with targets
print(target_tensor)

tensor([17.5920,  9.1302, 13.6620, 11.8540,  6.8233, 11.8860,  4.3483, 12.0000,
         6.5987,  3.8166,  3.2522, 15.5050,  3.1551,  7.2258,  0.7162,  3.5129,
         5.3048,  0.5608,  3.6518,  5.3893,  3.1386, 21.7670,  4.2630,  5.1875,
         3.0825, 22.6380, 13.5010,  7.0467, 14.6920, 24.1470, -1.2200,  5.9966,
        12.1340,  1.8495,  6.5426,  4.5623,  4.1164,  3.3928, 10.1170,  5.4974,
         0.5566,  3.9115,  5.3854,  2.4406,  6.7318,  1.0463,  5.1337,  1.8440,
         8.0043,  1.0179,  6.7504,  1.8396,  4.2885,  4.9981,  1.4233, -1.4211,
         2.4756,  4.6042,  3.9624,  5.4141,  5.1694, -0.7428, 17.9290, 12.0540,
        17.0540,  4.8852,  5.7442,  7.7754,  1.0173, 20.9920,  6.6799,  4.0259,
         1.2784,  3.3411, -2.6807,  0.2968,  3.8845,  5.7014,  6.7526,  2.0576,
         0.4795,  0.2042,  0.6786,  7.5435,  5.3436,  4.2415,  6.7981,  0.9270,
         0.1520,  2.8214,  1.8451,  4.2959,  7.2029,  1.9869,  0.1445,  9.0551,
         0.6170], dtype=torch.float64)


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [14]:
# MSE loss

def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [15]:
# Compute loss

loss = mse(preds, target_tensor)
print(loss)

tensor(923.4683, dtype=torch.float64, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [16]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [17]:
# Gradients for weights

print(w)
print(w.grad)

tensor([[-0.4477,  0.2369,  0.8880, -0.6696,  1.2064,  2.8565,  1.1719, -0.8014,
          1.1286, -0.0455, -0.0478, -2.0641,  0.5735,  0.2469,  0.3314, -1.4978,
          0.1663,  0.0702,  0.7890, -1.4708, -1.1601,  1.3935,  0.8092,  0.5814,
          0.8074,  0.5682,  0.0778,  0.0995, -0.2982,  1.1042, -0.5931,  0.4432,
         -1.1230,  0.2500,  0.5983,  0.4197, -1.3638, -0.0213,  0.3727, -1.1828,
          1.1756,  0.0996, -0.5443,  0.2518, -0.0046, -0.2861, -2.1497,  0.9983,
         -0.0464,  0.6934, -0.3807,  0.9900, -0.9996,  1.4707,  1.4489, -0.3441,
          0.3610,  0.5304,  0.9306,  2.6264, -3.3373, -0.4624, -0.8587, -0.6527,
         -0.7350,  0.4743, -1.3963,  0.4141, -0.2004,  0.2807,  0.5454, -0.4717,
         -0.1470, -3.1968, -1.0490, -0.8867, -1.8133,  0.9407, -0.5100, -1.2234,
         -1.9143, -0.3672, -1.1506,  0.3424, -2.7108,  0.7486, -0.3076, -0.2460,
         -1.4921, -0.2684, -1.3765,  0.7667, -0.3086, -0.3279,  2.0052,  0.9371,
          2.0364]], requires

## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [18]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [19]:
print(w)
print(b)

tensor([[-4.4408e-01,  2.4025e-01,  8.9309e-01, -6.6542e-01,  1.2099e+00,
          2.8615e+00,  1.1764e+00, -7.9624e-01,  1.1325e+00, -4.2436e-02,
         -4.4422e-02, -2.0557e+00,  5.7694e-01,  2.5196e-01,  3.3480e-01,
         -1.4946e+00,  1.7015e-01,  7.3248e-02,  7.9283e-01, -1.4665e+00,
         -1.1564e+00,  1.4056e+00,  8.1253e-01,  5.8522e-01,  8.1072e-01,
          5.7952e-01,  8.5431e-02,  1.0604e-01, -2.9033e-01,  1.1175e+00,
         -5.8993e-01,  4.4718e-01, -1.1174e+00,  2.5348e-01,  6.0321e-01,
          4.2440e-01, -1.3589e+00, -1.7981e-02,  3.8037e-01, -1.1790e+00,
          1.1788e+00,  1.0371e-01, -5.3734e-01,  2.5529e-01,  1.1607e-04,
         -2.8190e-01, -2.1466e+00,  1.0017e+00, -3.9451e-02,  6.9670e-01,
         -3.7621e-01,  9.9318e-01, -9.9519e-01,  1.4753e+00,  1.4527e+00,
         -3.4025e-01,  3.6474e-01,  5.3378e-01,  9.3612e-01,  2.6321e+00,
         -3.3321e+00, -4.5926e-01, -8.4598e-01, -6.4377e-01, -7.2365e-01,
          4.7861e-01, -1.3913e+00,  4.

In [20]:
#Calculating loss

preds = model(inputs_tensor.float())
loss = mse(preds, target_tensor)
print(loss)

tensor(663.8138, dtype=torch.float64, grad_fn=<DivBackward0>)


We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [21]:
# Let's verify that the loss is actually lower

loss = mse(preds, target_tensor)
print(loss)

tensor(663.8138, dtype=torch.float64, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [22]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [23]:
# Generate predictions

preds = model(inputs_tensor.float())
print(preds)

tensor([[-18.7101, -18.5175, -17.0732, -19.5699, -17.2516, -19.0605, -19.3314,
         -20.6843, -20.8767, -20.5308, -18.6260, -19.6480, -17.8551, -20.9291,
         -18.7599, -19.6920, -18.9201, -18.3685, -19.3798, -19.4469, -20.3929,
         -20.5352, -18.6969, -20.3125, -17.7054, -17.9344, -19.2941, -19.1825,
         -21.1583, -18.9144, -19.8429, -20.6468, -19.7857, -19.3216, -19.9045,
         -16.8917, -21.6860, -20.5821, -19.5389, -19.8215, -18.0407, -19.5850,
         -20.3511, -18.9968, -19.6597, -17.9920, -18.5073, -18.2751, -17.4684,
         -19.3668, -19.6058, -18.9993, -20.0209, -17.9209, -18.9613, -19.4565,
         -19.5811, -19.3081, -18.7320, -18.9308, -21.5113, -20.7141, -19.8075,
         -19.5520, -17.6253, -18.2357, -19.5698, -19.6642, -20.0217, -19.9722,
         -21.1465, -17.8763, -20.1125, -18.4213, -20.9014, -19.3693, -19.1862,
         -20.9661, -19.0970, -18.6908, -20.2300, -18.2976, -17.6033, -17.7353,
         -19.6502, -18.9851, -19.1497, -20.7751, -19

In [24]:
# Calculate the loss

loss = mse(preds, target_tensor)
print(loss)

tensor(663.8138, dtype=torch.float64, grad_fn=<DivBackward0>)


In [25]:
# Compute gradients

loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -307.4328,  -278.1291,  -428.6178,  -352.3696,  -294.8389,  -421.7900,
          -376.1789,  -431.6115,  -326.3565,  -254.3248,  -287.3368,  -712.6689,
          -288.5092,  -423.0730,  -283.8147,  -270.6673,  -320.2784,  -258.1236,
          -323.5086,  -355.7710,  -311.4077, -1019.8956,  -276.2372,  -318.3010,
          -280.0008,  -953.2275,  -645.4474,  -551.3071,  -662.9572, -1117.1554,
          -264.2772,  -331.5491,  -465.3280,  -296.4490,  -413.1457,  -399.1731,
          -407.3494,  -282.0839,  -645.8500,  -319.6746,  -272.0510,  -346.2966,
          -589.0941,  -290.5067,  -393.7038,  -356.8930,  -255.1097,  -291.9005,
          -588.6915,  -278.8285,  -379.3891,  -267.0597,  -373.5374,  -382.5539,
          -318.6381,  -319.9514,  -315.6896,  -283.7644,  -468.4475,  -475.6628,
          -444.0546,  -260.5992, -1070.6638,  -750.1038,  -953.9319,  -363.1875,
          -417.3723,  -515.0297,  -276.7052, -1023.4679,  -509.9981,  -369.0392,
          -302.2050,  -363.5

Let's update the weights and biases using the gradients computed above.

In [26]:
# Adjust weights & reset gradients

with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [27]:
print(w)
print(b)

tensor([[-0.4410,  0.2430,  0.8974, -0.6619,  1.2128,  2.8657,  1.1801, -0.7919,
          1.1358, -0.0399, -0.0415, -2.0485,  0.5798,  0.2562,  0.3376, -1.4919,
          0.1734,  0.0758,  0.7961, -1.4630, -1.1533,  1.4158,  0.8153,  0.5884,
          0.8135,  0.5891,  0.0919,  0.1116, -0.2837,  1.1286, -0.5873,  0.4505,
         -1.1128,  0.2564,  0.6073,  0.4284, -1.3548, -0.0152,  0.3868, -1.1758,
          1.1816,  0.1072, -0.5315,  0.2582,  0.0041, -0.2783, -2.1441,  1.0047,
         -0.0336,  0.6995, -0.3724,  0.9959, -0.9915,  1.4791,  1.4559, -0.3371,
          0.3679,  0.5366,  0.9408,  2.6368, -3.3276, -0.4567, -0.8353, -0.6363,
         -0.7141,  0.4822, -1.3871,  0.4253, -0.1944,  0.3030,  0.5566, -0.4637,
         -0.1404, -3.1889, -1.0435, -0.8795, -1.8050,  0.9462, -0.4987, -1.2178,
         -1.9080, -0.3615, -1.1436,  0.3531, -2.7036,  0.7579, -0.2975, -0.2394,
         -1.4861, -0.2629, -1.3703,  0.7751, -0.3022, -0.3220,  2.0143,  0.9518,
          2.0424]], requires

With the new weights and biases, the model should have a lower loss.

In [28]:
# Calculate loss

preds = model(inputs_tensor.float())
loss = mse(preds, target_tensor)
print(loss)

tensor(479.6952, dtype=torch.float64, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [29]:
# Train for 100 epochs

for i in range(100):
    preds = model(inputs_tensor.float())
    loss = mse(preds, target_tensor)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [30]:
# Calculate loss

preds = model(inputs_tensor.float())
loss = mse(preds, target_tensor)
print(loss)

tensor(30.8997, dtype=torch.float64, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [31]:
# Predictions
preds

tensor([[6.4479, 6.6403, 8.0846, 5.5880, 7.9061, 6.0974, 5.8263, 4.4736, 4.2811,
         4.6270, 6.5316, 5.5100, 7.3026, 4.2287, 6.3977, 5.4656, 6.2376, 6.7891,
         5.7778, 5.7108, 4.7648, 4.6229, 6.4608, 4.8452, 7.4523, 7.2236, 5.8638,
         5.9752, 3.9997, 6.2437, 5.3147, 4.5109, 5.3722, 5.8360, 5.2533, 8.2660,
         3.4718, 4.5756, 5.6189, 5.3362, 7.1169, 5.5727, 4.8066, 6.1608, 5.4980,
         7.1656, 6.6504, 6.8825, 7.6893, 5.7908, 5.5519, 6.1584, 5.1368, 7.2367,
         6.1963, 5.7011, 5.5765, 5.8496, 6.4257, 6.2270, 3.6464, 4.4436, 5.3505,
         5.6059, 7.5327, 6.9220, 5.5879, 5.4935, 5.1359, 5.1858, 4.0113, 7.2813,
         5.0452, 6.7364, 4.2562, 5.7883, 5.9715, 4.1916, 6.0607, 6.4669, 4.9276,
         6.8600, 7.5542, 7.4225, 5.5075, 6.1726, 6.0081, 4.3826, 5.7461, 6.7051,
         6.5000, 6.6659, 5.4433, 4.4512, 5.6787, 6.0191, 5.3100]],
       grad_fn=<AddBackward0>)

In [32]:
# Targets
targets

0      17.592
1      9.1302
2      13.662
3      11.854
4      6.8233
       ...   
92     7.2029
93     1.9869
94    0.14454
95     9.0551
96    0.61705
Name: 1, Length: 97, dtype: object

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [33]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [34]:
# Input(Population)
inputs = torch.from_numpy(input_arr)

# Targets(Profit)
targets = torch.from_numpy(target_arr)

In [35]:
print(inputs)
print(targets)

tensor([ 6.1101,  5.5277,  8.5186,  7.0032,  5.8598,  8.3829,  7.4764,  8.5781,
         6.4862,  5.0546,  5.7107, 14.1640,  5.7340,  8.4084,  5.6407,  5.3794,
         6.3654,  5.1301,  6.4296,  7.0708,  6.1891, 20.2700,  5.4901,  6.3261,
         5.5649, 18.9450, 12.8280, 10.9570, 13.1760, 22.2030,  5.2524,  6.5894,
         9.2482,  5.8918,  8.2111,  7.9334,  8.0959,  5.6063, 12.8360,  6.3534,
         5.4069,  6.8825, 11.7080,  5.7737,  7.8247,  7.0931,  5.0702,  5.8014,
        11.7000,  5.5416,  7.5402,  5.3077,  7.4239,  7.6031,  6.3328,  6.3589,
         6.2742,  5.6397,  9.3102,  9.4536,  8.8254,  5.1793, 21.2790, 14.9080,
        18.9590,  7.2182,  8.2951, 10.2360,  5.4994, 20.3410, 10.1360,  7.3345,
         6.0062,  7.2259,  5.0269,  6.5479,  7.5386,  5.0365, 10.2740,  5.1077,
         5.7292,  5.1884,  6.3557,  9.7687,  6.5159,  8.5172,  9.1802,  6.0020,
         5.5204,  5.0594,  5.7077,  7.6366,  5.8707,  5.3054,  8.2934, 13.3940,
         5.4369], dtype=torch.float64)
t

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [36]:
from torch.utils.data import TensorDataset

In [37]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:2]

(tensor([6.1101, 5.5277], dtype=torch.float64),
 tensor([17.5920,  9.1302], dtype=torch.float64))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [38]:
from torch.utils.data import DataLoader

In [39]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [40]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([14.9080,  5.6063, 13.3940,  6.5159,  7.4764], dtype=torch.float64)
tensor([12.0540,  3.3928,  9.0551,  5.3436,  4.3483], dtype=torch.float64)


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [43]:
# Define model
model = nn.Linear(97,1)

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [42]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0142,  0.0713,  0.0655, -0.0983, -0.0501,  0.0266, -0.0782, -0.0322,
           0.0842,  0.0223,  0.0057,  0.0071,  0.0111,  0.0042, -0.0471,  0.0770,
           0.0619, -0.0949,  0.0121,  0.0911, -0.0886, -0.0155,  0.0288,  0.0766,
          -0.0944,  0.0812,  0.0998, -0.0375,  0.0121,  0.0822,  0.0337,  0.0977,
          -0.0980,  0.0195, -0.0020, -0.0932,  0.0711, -0.0602, -0.0718,  0.0329,
          -0.0477, -0.0884,  0.0104,  0.0854, -0.0273, -0.0911,  0.0477, -0.0771,
          -0.0146, -0.0293, -0.0984, -0.0845, -0.0168,  0.0571,  0.0820, -0.0416,
           0.0139,  0.0054, -0.0907, -0.0880,  0.0461, -0.0985, -0.0187,  0.0484,
          -0.0987,  0.0812, -0.0921,  0.0675,  0.0150,  0.0680, -0.0627, -0.0399,
          -0.0304, -0.0637, -0.0401, -0.0280, -0.0393,  0.0718,  0.0866,  0.0380,
           0.0241,  0.0008, -0.0659,  0.0824,  0.0815,  0.0432,  0.0318, -0.0421,
           0.0584,  0.0986,  0.0241,  0.0980, -0.0932,  0.0978, -0.1006, -0

We can use the model to generate predictions in the same way as before.

In [44]:
# Generate predictions
preds = model(inputs.float()) 
print(preds)

tensor([2.4988], grad_fn=<AddBackward0>)


## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [45]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [46]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [48]:
loss = loss_fn(model(inputs.float()), targets)

C:\Users\sumed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Using a target size (torch.Size([97])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  """Entry point for launching an IPython kernel.


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [49]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [50]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [51]:
fit(100, model, loss_fn, opt, train_dl)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 97x1)

Let's generate predictions using our model and verify that they're close to our targets.

In [ ]:
# Generate predictions
preds = model(inputs.float())
preds

In [ ]:
# Compare with targets
targets

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 



## Commit and update the notebook

As a final step, we can record a new version of the notebook using the `jovian` library.

In [ ]:
import jovian

In [ ]:
jovian.commit(project='02-linear-regression')

Note that running `jovian.commit` a second time records a new version of your existing notebook. With Jovian.ml, you can avoid creating copies of your Jupyter notebooks and keep versions organized. Jovian also provides a visual diff ([example](https://jovian.ai/aakashns/keras-mnist-jovian/diff?base=8&remote=2)) so you can inspect what has changed between different versions:

![jovian-commenting](https://i.imgur.com/HF1cOVt.png)

## Exercises and Further Reading

We've covered the following topics in this tutorial:

- Introduction to linear regression and gradient descent
- Implementing a linear regression model using PyTorch tensors
- Training a linear regression model using the gradient descent algorithm
- Implementing gradient descent and linear regression using PyTorch built-in


Here are some resources for learning more about linear regression and gradient descent:

* An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: [Working with Images & Logistic Regression](https://jovian.ai/aakashns/03-logistic-regression).

## Questions for Review

Try answering the following questions to test your understanding of the topics covered in this notebook:

1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.
2. What are input and target variables in a dataset? Give an example.
3. What are weights and biases in a linear regression model?
4. How do you represent tabular data using PyTorch tensors?
5. Why do we create separate matrices for inputs and targets while training a linear regression model?
6. How do you determine the shape of the weights matrix & bias vector given some training data?
7. How do you create randomly initialized weights & biases with a given shape?
8. How is a linear regression model implemented using matrix operations? Explain with an example.
9. How do you generate predictions using a linear regression model?
10. Why are the predictions of a randomly initialized model different from the actual targets?
11. What is a loss function? What does the term “loss” signify?
12. What is mean squared error?
13. Write a function to calculate mean squared using model predictions and actual targets.
14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?
15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? What do its elements represent?
16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.
17. Suppose the derivative  of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?
18. Suppose the derivative  of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?
19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?
20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?
21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?
22. What is learning rate? Why is it important?
23. What is `torch.no_grad`?
24. Why do you reset gradients to zero after updating weights and biases?
25. What are the steps involved in training a linear regression model using gradient descent?
26. What is an epoch?
27. What is the benefit of training a model for multiple epochs?
28. How do you make predictions using a trained model?
29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.
30. What is `torch.nn`?
31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.
32. What is a data loader in PyTorch? Give an example.
33. How do you use a data loader to retrieve batches of data?
34. What are the benefits of shuffling the training data before creating batches?
35. What is the benefit of training in small batches instead of training with the entire dataset?
36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.
37. How do you see the weights and biases of a `nn.Linear` model?
38. What is the purpose of the `torch.nn.functional` module?
39. How do you compute mean squared error loss using a PyTorch built-in function?
40. What is an optimizer in PyTorch?
41. What is `torch.optim.SGD`? What does SGD stand for?
42. What are the inputs to a PyTorch optimizer? 
43. Give an example of creating an optimizer for training a linear regression model.
44. Write a function to train a `nn.Linear` model in batches using gradient descent.
45. How do you use a linear regression model to make predictions on previously unseen data? 
